<a href="https://colab.research.google.com/github/rhks13/DART/blob/main/%EB%AA%A8%ED%88%AC_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# **라이브러리 준비**

In [2]:
pip install opendartreader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install pykrx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.5/125.5 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 KB 11.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import OpenDartReader
from pykrx import stock
from tqdm import tqdm 
import re

# **1. 사업보고서 문서번호 추출**

## **1-1. OpenDartReader 활용 방식**

In [5]:
# OPEN DART 에서 API KEY를 받아온 후 your api key에 입력
api_key = '395efb1f679b81def9af0fa2cd9a1e8032ad2ed3'
dart = OpenDartReader(api_key)

In [6]:
# 티커 준비
tickers = stock.get_market_ticker_list(market='KOSPI')

In [7]:
# 예시 -> ticker[0] 조회 시작 지점부터의 공시정보 데이터 프레임
dart.list(tickers[0],start='2014-01-01')

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00365387,AJ네트웍스,095570,Y,기업설명회(IR)개최(안내공시),20230403800809,AJ네트웍스,20230403,유
1,00365387,AJ네트웍스,095570,Y,사외이사의선임ㆍ해임또는중도퇴임에관한신고,20230329001002,AJ네트웍스,20230329,
2,00365387,AJ네트웍스,095570,Y,대표이사(대표집행임원)변경(안내공시),20230329801723,AJ네트웍스,20230329,유
3,00365387,AJ네트웍스,095570,Y,정기주주총회결과,20230329801505,AJ네트웍스,20230329,유
4,00365387,AJ네트웍스,095570,Y,사업보고서 (2022.12),20230321001546,AJ네트웍스,20230321,
...,...,...,...,...,...,...,...,...,...
333,00365387,AJ네트웍스,095570,Y,증권발행실적보고서,20140509000056,AJ네트웍스,20140509,
334,00365387,AJ네트웍스,095570,Y,투자설명서,20140508000020,AJ네트웍스,20140508,
335,00365387,AJ네트웍스,095570,Y,[발행조건확정]증권신고서(채무증권),20140507001817,AJ네트웍스,20140508,
336,00365387,AJ네트웍스,095570,Y,철회신고서,20140409001399,AJ네트웍스,20140409,


## **1-2. 실제 추출 과정**



### [ 2014년 이후 사업보고서의 문서번호(rceptno)추출 ]



1.   조회시점을 2014-01-01로 설정

  1-1. 단순 조회시점을 설정해도 이전연도들의 사업보고서를 수정할 시 같이 검색됨

  1-2. 이에 함수를 통해 보고서의 제목에 포함된 연도가 2014보다 큰 항목들만 추출


2.   '사업보고서'라는 텍스트를 포함하는 제목의 문서만 추출

  2-1. 단순 '사업보고서'로 contains을 사용 => '사업보고서제출기한연장신고서'까지 함께 도출

  2-2. 이에 '사업보고서 \('를 활용해 추출



3. 모든 항목에 대해 적용







In [ ]:
# 모든 종목의 사업보고서 문서번호 추출 (rceptno 열)

def bools(x):
  if int(x[-8:-4])>=2014:
    return True
  else:
    return False

doc_inds = []
for i in tqdm(range(len(tickers))):
  try:
    df = dart.list(tickers[i],start='2014-01-01')
    df = df[df['report_nm'].str.contains('사업보고서 \(')] #사업보고서 뽑기위한 과정
    ind = list(df[df['report_nm'].apply(bools)]['rcept_no'])
    doc_inds.append(ind)
  except:
    doc_inds.append(['None']*10)

100%|██████████| 944/944 [1:04:29<00:00,  4.10s/it]


In [ ]:
# 데이터 프레임 생성을 위해 리스트의 len이 다르면 None을 append
for i in tqdm(range(len(doc_inds))):
    while len(doc_inds[i]) < 10:
        doc_inds[i].append(None)

100%|██████████| 944/944 [00:00<00:00, 138137.07it/s]


In [ ]:
# 문서번호가 정리된 dataframe[ind_df] 생성
ind_df=pd.DataFrame(doc_inds).T
corp_name=[stock.get_market_ticker_name(i) for i in tickers] 
ind_df.columns=corp_name

ind_df = ind_df.iloc[:9,:]
ind_df

,AJ네트웍스,AK홀딩스,BGF,BGF리테일,BNK금융지주,BYC,BYC우,CJ,CJ CGV,CJ4우(전환),...,효성화학,후성,휠라홀딩스,휴니드,휴비스,휴스틸,흥국화재,흥국화재2우B,흥국화재우,흥아해운
0,20230321001546,20230323001504,20230320000850,20230320000886,20230309000759,20230316001312,None,20230321001305,20230320001054,None,...,20230308000769,20230321000387,20230321000775,20230313000533,20230321001473,20230322001091,20230316001454,None,None,20230320001057
1,20220805000318,20220321001495,20220321000840,20220321001041,20220317001180,20220317001262,None,20220321001352,20220321001182,None,...,20220308000678,20220321001474,20220812000706,20220308000872,20220316000822,20220323001079,20220317001158,None,None,20220816000435
2,20210323000506,20210323000581,20210317000812,20210317000292,20210517001446,20210318001376,None,20210323001025,20210319001272,None,...,20210310001197,20210322000692,20210319000773,20210317001215,20210420000253,20210323001094,20210318001280,None,None,20210729000451
3,20200330003824,20200330003219,20210317000625,20200330003848,20200330004566,20200330003485,None,20200330004294,20200330003232,None,...,20200330004578,20200330004633,20200327001877,20200330003158,20200330002584,20200330004131,20210114000331,None,None,20200409002668
4,20190531001150,20190401004433,20210317000601,20190401003766,20190401004838,20190401003573,None,20190329004141,20190702000269,None,...,20190401004073,20190401002191,20190614000418,20191016000377,20190329002228,20190401004479,20190401001678,None,None,20191120000137
5,20180330003492,20180330002417,20180402002991,20180402003237,20180629000837,20180402004873,None,20180330003087,20180330003675,None,...,None,20180402000932,20180419000484,20191016000372,20180330003411,20180402005126,20180330000905,None,None,20180330001430
6,20170331002568,20170331003282,20170404001648,None,20170331005308,20171226000456,None,20170331004642,20170331004598,None,...,None,20170607000125,20170526000555,20170331003771,20170331000466,20170331004785,20170331001148,None,None,20170331003984
7,20160524000353,20160330003602,20160330002688,None,20160330004092,20160330003578,None,20160330003575,20160330004050,None,...,None,20160330000319,20160516002748,20160330001845,20160329001187,20160330001561,20160330003906,None,None,20160329000856
8,20150605000363,20150605000083,20150603000166,None,20150331003821,20150331003770,None,20150515002123,20150331004218,None,...,None,20150331002792,20150331003731,20150331000877,20150514003525,20150331003907,20150330001115,None,None,20150331000251


In [ ]:
# 결측값이 존재하는 회사 제거

ind_df.dropna(axis=1,inplace=True)

def isn(x):
  if x =='None':
    return None
  else:
    return x

ind_df=ind_df.applymap(isn).dropna(axis=1)
ind_df

,AJ네트웍스,AK홀딩스,BGF,BNK금융지주,BYC,CJ,CJ CGV,CJ대한통운,CJ씨푸드,CJ제일제당,...,황금에스티,효성,효성ITX,후성,휠라홀딩스,휴니드,휴비스,휴스틸,흥국화재,흥아해운
0,20230321001546,20230323001504,20230320000850,20230309000759,20230316001312,20230321001305,20230320001054,20230317000857,20230320000740,20230320000712,...,20230320000391,20230309000903,20230315001063,20230321000387,20230321000775,20230313000533,20230321001473,20230322001091,20230316001454,20230320001057
1,20220805000318,20220321001495,20220321000840,20220317001180,20220317001262,20220321001352,20220321001182,20220318000927,20220321000834,20220321001115,...,20220321000623,20220310001203,20220315001007,20220321001474,20220812000706,20220308000872,20220316000822,20220323001079,20220317001158,20220816000435
2,20210323000506,20210323000581,20210317000812,20210517001446,20210318001376,20210323001025,20210319001272,20210319001047,20210319000697,20210318001302,...,20220225000997,20210324000507,20210315000907,20210322000692,20210319000773,20210317001215,20210420000253,20210323001094,20210318001280,20210729000451
3,20200330003824,20200330003219,20210317000625,20200330004566,20200330003485,20200330004294,20200330003232,20200330003195,20200330002361,20200330003752,...,20220224003906,20200330004774,20200330004238,20200330004633,20200327001877,20200330003158,20200330002584,20200330004131,20210114000331,20200409002668
4,20190531001150,20190401004433,20210317000601,20190401004838,20190401003573,20190329004141,20190702000269,20191002000600,20190401003597,20190401004092,...,20190329004157,20190401002305,20190401004046,20190401002191,20190614000418,20191016000377,20190329002228,20190401004479,20190401001678,20191120000137
5,20180330003492,20180330002417,20180402002991,20180629000837,20180402004873,20180330003087,20180330003675,20180402002335,20180402004301,20180406002292,...,20180330001418,20180326000377,20180402003873,20180402000932,20180419000484,20191016000372,20180330003411,20180402005126,20180330000905,20180330001430
6,20170331002568,20170331003282,20170404001648,20170331005308,20171226000456,20170331004642,20170331004598,20170331005048,20170331004236,20170331004895,...,20170331000866,20170331004981,20170331003412,20170607000125,20170526000555,20170331003771,20170331000466,20170331004785,20170331001148,20170331003984
7,20160524000353,20160330003602,20160330002688,20160330004092,20160330003578,20160330003575,20160330004050,20160330003057,20160330004074,20160330003590,...,20160331000001,20160330004322,20160330003016,20160330000319,20160516002748,20160330001845,20160329001187,20160330001561,20160330003906,20160329000856
8,20150605000363,20150605000083,20150603000166,20150331003821,20150331003770,20150515002123,20150331004218,20150331002584,20150514001845,20150515002841,...,20150331002278,20150519000168,20150331003856,20150331002792,20150331003731,20150331000877,20150514003525,20150331003907,20150330001115,20150331000251


In [ ]:
# ind_df.to_csv('/content/gdrive/MyDrive/DART/모투/23-1 모투/data/최종ind_0329')

# **2. 사업보고서 텍스트 전처리**

In [ ]:
# ind_df = pd.read_csv('/content/gdrive/MyDrive/DART/모투/23-1 모투/data/최종ind_0329', index_col =0)

In [ ]:
# OpenDartReader로 문서번호를 입력해 텍스트 받아오기 
dart.document(ind_df.iloc[0,0])[:4000]

'<?xml version="1.0" encoding="utf-8"?>\r\n\r\n\r\n<DOCUMENT xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="dart3.xsd">\n\n<DOCUMENT-NAME ACODE="11011">사업보고서</DOCUMENT-NAME>\n<FORMULA-VERSION ADATE="20230220">5.0</FORMULA-VERSION>\n<COMPANY-NAME AREGCIK="00365387">AJ네트웍스(주)</COMPANY-NAME>\n\n<SUMMARY>\n<EXTRACTION ACODE="LINK_FLAG" AFEATURE="BOTH">C</EXTRACTION>\n<EXTRACTION ACODE="FIN_TYPE" AFEATURE="BOTH">A</EXTRACTION>\n<EXTRACTION ACODE="CRP_RGS_NO_TEMP" AFEATURE="BOTH">110111-1874654</EXTRACTION>\n</SUMMARY>\n\n\n<BODY>\n\n<COVER>\n<COVER-TITLE ATOC="Y" AASSOCNOTE="COVER">사 업 보 고 서</COVER-TITLE>\n<P></P>\n<P USERMARK="F-16">                                    (제 23 기)</P>\n\n<TABLE-GROUP ACLASS="COVER" ADELETETABLE="N">\n\n<TABLE ACLASS="EXTRACTION" AFIXTABLE="Y" WIDTH="600" BORDER="0">\n\n<COLGROUP>\n<COL WIDTH="209"></COL>\n<COL WIDTH="153"></COL>\n<COL WIDTH="211"></COL>\n</COLGROUP>\n\n<TBODY>\n\n<TR ACOPY="N" ADELETE="N">\n<TD CLASS="NORM

### **[ 수행방식 ]**

1. 목차의 헤더를 이용해 내용 추출

* ‘IV. 이사의 경영진단 및 분석의견'과 'V. 회계감사인의감사의견 등’의 사이에 있는 xml만 추출
*   기재정정된 경우 존재 => 보고서의 초반부에 수정내용 언급하며 헤더가 한번 더 작성된 경우
*   사업보고서의 양식 변화 -> 2018년 이전 사업보고서의 목차에서는 'V. 이사의 경영진단 및 분석의견'으로 기재됨


> => 조건에 맞게 헤더를 색인하며 추출할 데이터의 시작인덱스를 찾음




2. 불필요한 내용 &  항목 삭제



*   문서마다 반복되지만 불필요한 정규 표현식 활용해 전처리
*   테이블, 단위, 연결재무제표 관련 내용 삭제
*   태그 삭제
*   괄호 안의 내용 삭제
*   3칸 이상의 공백을 1칸으로 대체





















In [ ]:

def context_21(corpus, start_cont, end_cont):
  temp = corpus.find('IV. 이사의 경영진단 및 분석의견')+1
  jump = corpus[temp:].find('IV. 이사의 경영진단 및 분석의견')
  if temp == 0: # 2018년 이전에는 V
    temp = corpus.find('V. 이사의 경영진단 및 분석의견')+1 
    jump = corpus[temp:].find('V. 이사의 경영진단 및 분석의견')
    start_cont = 'V. 이사의 경영진단 및 분석의견'
    end_cont = 'VI. 이사회 등 회사의 기관에 관한 사항'
  if jump == -1:
    jump=0

  start_ind = corpus.find(start_cont, jump)
  end_ind = corpus.find(end_cont, jump)
  raw_cont = corpus[start_ind:end_ind]
  
  # 필요없는 내용들 삭제
  raw_cont = re.sub("\n",' ',raw_cont)
  raw_cont = re.sub("(단위 : 백만원)",' ',raw_cont)
  raw_cont = re.sub("- 상기 연결재무제표는 한국채택국제회계기준 작성기준에 따라 작성되었습니다.",' ',raw_cont)
  
  # 테이블, 태그 삭제
  clean_table = re.compile("<TABLE.*?TABLE>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});") #테이블 삭제
  raw_cont = re.sub(clean_table,'', raw_cont)
  clean_tag = re.compile("<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});") #태그 삭제
  fin_cont = re.sub(clean_tag,'', raw_cont)

  # 대괄호, 소괄호, 괄호 사이의 단어 제거
  fin_cont = re.sub(r'\[[^\]]*\]', '', fin_cont)
  fin_cont = re.sub(r'\([^\)]*\)', '', fin_cont)
  fin_cont = re.sub(r'\{[^\}]*\}', '', fin_cont)
  # click 제거
  fin_cont = re.sub('◆click◆', '',fin_cont)
  # 3칸 이상 공백을 1칸으로 대체
  fin_cont = re.sub(r'\s{3,}', ' ', fin_cont)
  return fin_cont

In [ ]:
# context함수 적용시 결과 예시
context_21(corpus,'IV. 이사의 경영진단 및 분석의견','V. 회계감사인의 감사의견 등')[22:]

' 1. 예측정보에 대한 주의사항  당사가 동 사업보고서에서 경영성과 등에 대해 분석한 의견과 미래에 발생할 것으로 예상, 예측한 활동, 사건 또는 현상은 공시서류 작성시점의 사건 및 재무성과에 대하여 회사의 견해를 반영하고 있습니다. 동 예측정보는 미래 사업환경과 관련된 다양한가정에 기초하고 있으며, 동 가정에 근거한 예측정보는 실제 결과 간에 중요한 차이를 초래할 수 있는 위험, 불확실성 및 기타 요인을 포함하고 있습니다. 이러한 중요한차이를 초래할 수 있는 요인에는 회사 내부경영과 관련된 요인과 외부환경에 관한 요인이 포함되어 있으며, 이에 한하지 않습니다.  또한 향후 전망은 현재의 시장상황과 회사의 경영방향등을 고려한 것으로서 향후 시장환경의 변화와 전략수정 등에 따라 달라질 수 있습니다.  당사는 동 예측정보 작성시점 이후에 발생하는 위험 또는 불확실성을 반영하기 위해 예측정보에 기재한 사항을 수정하는 정정보고서를 공시할 의무는 없습니다.     2. 개요               ■ 지주회사 및 주요종속회사     당사는 2017년 6월 8일자 이사회에서 \'지주회사 부문\'과 \'편의점 연쇄화사업부문\'을 인적분할하는 결의를 하였으며, 2017년 9월 28일자 주주총회에서 동 결의안을 원안대로 승인하였습니다. 이에 따라 당사는 2017년 11월 1일을 분할기일로 하여 인적분할의 방법으로 주식회사 비지에프리테일을 신설하고, 분할존속회사인 당사의 명칭을 주식회사 비지에프로 변경하였습니다. 당사는 2018년 3월 9일 기준으로 \'독점규제 및 공정거래에 관한 법률\' 제2조 1의 2호 및 동법 시행령 제2조에 따른 지주회사 요건을 충족하였으며 공정거래위원회로부터 18년 3월 9일 현재 지주회사의 기준을 충족하고 있다는 내용의 심사결과 통지서를 18년 7월 16일자로 접수하였습니다.  이를 통해 기업지배구조의 투명성을 확립하고 궁극적으로 기업가치와 주주가치를 제고하고자 합니다.    지주회사로써 당사의 주요 수익은 브랜드 사용수익, 업무용역제공을 통한 

In [ ]:
# 전체 데이터프레임(ind_df)에 context를 적용시키기 위한 함수
def fin_text_21(doc_ind):
  try:
    corpus=dart.document(doc_ind)
    isa = context_21(corpus,'IV. 이사의 경영진단 및 분석의견','V. 회계감사인의 감사의견 등')[22:]
    return isa
  except:
    return ''

In [ ]:
fin_text_21('20180330003492')

'가 동 사업보고서에서 미래에 발생할 것으로 예상예측한 활동, 사건 또는 현상은 당해 공시서류 작성시점의 사건 및 재무성과에 대하여 회사의 견해를 반영한 것입니다. 동 예측정보는 미래 사업환경과 관련된 다양한 가정에 기초하고 있으며, 동 가정들은 결과적으로 부정확한 것으로 판명될 수도 있습니다. 또한, 이러한 가정들에는 예측정보에서 기재한 예상치와 실제 결과 간에 중요한 차이를 초래할 수 있는 위험, 불확실성 및 기타요인을 포함하고 있습니다. 이러한 중요한 차이를 초래할 수 있는 요인에는 회사 내부경영과 관련된 요인과 외부환경에 관한 요인이 포함되어 있으며, 이에 한하지 않습니다.당사는 동 예측정보 작성시점 이후에 발생하는 위험 또는 불확실성을 반영하기 위하여 예측정보에 기재한 사항을 수정하는 정정보고서를 공시할 의무는 없습니다. 결론적으로, 동 사업보고서상에 회사가 예상한 결과 또는 사항이 실현되거나 회사가 당초에 예상한 영향이 발생한다는 확신을 제공할 수 없습니다. 동 보고서에 기재된 예측정보는 동 보고서 작성시점을 기준으로 작성한 것이며, 회사가 이러한 위험요인이나 예측정보를 업데이트할 예정이 없음을 유의하시기 바랍니다.2. 개요 2017년은 국내외 혼란한 시장환경 속에서도 지속 가능한 성장을 추구하고 회사의 발전을 위해 각고의 노력을 기울였습니다. 당사는 내부적으로는 선택과 집중, 외부적으로 시장확장을 통해 2017년 한해를 훌륭히 마무리하였습니다.  현재 렌탈 시장은 소규모 업체가 영업을 하고 있는 상태이며, 자산 관리 노하우, 영업능력, 자금력, 채권회수능력을 겸비한 종합렌탈사는 당사를 포함하여 소수업체만이 사업을 영위하고 있습니다. 이는 렌탈노하우에 대한 진입장벽이 높은 것을 의미합니다. 또한 빠르게 변화하는 경영환경의 특성상 기업은 구매장비의 진부화 위험을 피하기 위하여 지속적으로 렌탈 비중을 확대할 것으로 보입니다.당사는 종합렌탈사로서 체계적인 프로세스를 갖추고, 렌탈 영업외 유통, 생산, 리퍼비쉬, 렌탈과 관련된 모든 산업과 연관할 수 있습니

In [ ]:
fin_df=ind_df.applymap(fin_text_21)
fin_df

,AJ네트웍스,AK홀딩스,BGF,BNK금융지주,BYC,CJ,CJ CGV,CJ대한통운,CJ씨푸드,CJ제일제당,...,황금에스티,효성,효성ITX,후성,휠라홀딩스,휴니드,휴비스,휴스틸,흥국화재,흥아해운
0,예측정보에 대한 주의사항 이 자료는 미래에 대한 예측정보를 포함하고 있습니다. 당...,"예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상, 예측...",예측정보에 대한 주의사항 당사가 동 사업보고서에서 경영성과 등에 대해 분석한 의견...,"예측정보에 대한 주의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상, 예...",예측정보에 대한 주의사항 당사가 이사의 경영진단 의견서에서 미래에 발생할 것으로 ...,"예측정보에 대한 주의사항본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 추...","자료는 회사의 재무상황, 영업실적 및 사업에 관련한 추정, 계획, 예상 등 '예측정...",예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상 예측한...,"예측정보에 대한 주의사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 ...","예측정보에 대한 주의 사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한...",...,의 당기와 전기의 경영성과를 비교하여 증감내역 및 원인을 진단하였습니다. 1. 예측...,예측정보에 대한 주의사항 회사가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예측...,예측정보에 대한 주의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상 ·...,예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상 ·예측...,예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예측한...,". 예측정보에 대한 주의사항 아래 분석의견은 미래에 대한 ""예측정보""를 포함하고 있...",예측정보에 대한 주의사항 당사가 본 경영진단의견서에서 미래에 발생할 것으로 예측...,예측정보에 대한 주의사항 : 당사가 동 사업보고서에서 미래에 발생할 것으로 예상...,. 예측정보에 대한 유의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예...,. 예측정보에 대한 주의사항당사가 본 경영진단 및 분석의견에서 미래에 발생할 것으로...
1,예측정보에 대한 주의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상예측한...,"예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상, 예측...",예 측 정보에 대한 주의사항 당사가 동 사업보고서에서 경영성과 등에 대해 분석한...,"예측정보에 대한 주의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상, 예...",예측정보에 대한 주의사항 당사가 이사의 경영진단 의견서에서 미래에 발생할 것으로 ...,"예측정보에 대한 주의사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 ...","예측정보에 대한 주의사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 ...",예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상 예측한...,"예측정보에 대한 주의사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 ...","예측정보에 대한 주의 사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한...",...,의 당기와 전기의 경영성과를 비교하여 증감내역 및 원인을 진단하였습니다. 1. 예측...,예측정보에 대한 주의사항 회사가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예측...,예측정보에 대한 주의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상 ·예...,예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상 ·예측...,예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예측한...,". 예측정보에 대한 주의사항 아래 분석의견은 미래에 대한 ""예측정보""를 포함하고 있...",예측정보에 대한 주의사항 당사가 본 경영진단의견서에서 미래에 발생할 것으로 예측하...,예측정보에 대한 주의사항 : 당사가 동 사업보고서에서 미래에 발생할 것으로 예상...,,예측정보에 대한 주의사항당사가 본 경영진단 및 분석의견에서 미래에 발생할 것으로 ...
2,"가 동 사업보고서에서 미래에 발생할 것으로 예상예측한 활동, 사건 또는 현상은 당해...","예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상, 예측...",예 측 정보에 대한 주의사항 당사가 동 사업보고서에서 경영성과 등에 대해 분석한...,,예측정보에 대한 주의사항 당사가 이사의 경영진단 의견서에서 미래에 발생할 것으로 ...,"예측정보에 대한 주의사항본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 추...","예측정보에 대한 주의사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 ...",예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상 예측한...,"예측정보에 대한 주의사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 ...",". 예측정보에 대한 주의 사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련...",...,,예측정보에 대한 주의사항 회사가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예측...,예측정보에 대한 주의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상 ·...,예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상 ·예측...,"가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예측한 활동, 사건 또는 현상은 당...",". 예측정보에 대한 주의사항 아래 분석의견은 미래에 대한 ""예측정보""를 포함하고 있...",예측정보에 대한 주의사항 당사가 본 경영진단의견서에서 미래에 발생할 것으로 예측...,예측정보에 대한 주의사항 : 당사가 동 사업보고서에서 미래에 발생할 것으로 예상...,. 예측정보에 대한 유의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예...,--------------------- 267 V. 감사인의 감사의견 등 -----...
3,"가 동 사업보고서에서 미래에 발생할 것으로 예상예측한 활동, 사건 또는 현상은 당해...","예측정보에 대한 주의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상, 예...",예측정보에 대한 주의사항 당사가 동 사업보고서에서 경영성과 등에 대해 분석한 의...,"예측정보에 대한 주의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상, 예...",예측정보에 대한 주의사항 당사가 이사의 경영진단 의견서에서 미래에 발생할 것으로 ...,"예측정보에 대한 주의사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 ...","예측정보에 대한 주의사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 ...",예측정보에 대한 주의사항당사가 동 사업보고서에서 미래에 발생할 것으로 예상 예측한...,"예측정보에 대한 주의사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련한 ...",". 예측정보에 대한 주의 사항 본 자료는 회사의 재무상황, 영업실적 및 사업에 관련...",...,의 당기와 전기의 경영성과를 비교하여 증감내역 및 원인을 진단하였습니다. 1. 예측...,에측정보에 대한 주의사항회사가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예측한...,예측정보에 대한 주의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상 ·...,예측정보에 대한 주의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상 ·예...,"가 동 사업보고서에서 미래에 발생할 것으로 예상 ·예측한 활동, 사건 또는 현상은,...",". 예측정보에 대한 주의사항 아래 분석의견은 미래에 대한 ""예측정보""를 포함하고 있...",예측정보에 대한 주의사항 당사가 본 경영진단의견서에서 미래에 발생할 것으로 예측...,예측정보에 대한 주의사항 : 당사가 동 사업보고서에서 미래에 발생할 것으로 예상...,예측정보에 대한 유의사항 당사가 동 사업보고서에서 미래에 발생할 것으로 예상ㆍ예측...,. 예측정보에 대한 주의사항당사가 본 경영진단 및 분석의견에서 미래에 발생할 것으로...
4,"사가 동 사업보고서에

In [ ]:
#fin_df.to_csv('/content/gdrive/MyDrive/DART/모투/23-1 모투/data/최종_텍스트.csv')